# Import required packages and load dataset

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials

# If you are running Databricks Runtime for Machine Learning, `mlflow` is already installed and you can skip the following line. 
import mlflow

- The Iris dataset is a widely used dataset in the field of machine learning and data analysis. It is named after the Iris flower plant and was introduced by the British statistician and biologist Ronald Fisher in 1936. The dataset is frequently used as a beginner's dataset to learn and practice various classification algorithms.

- The Iris dataset consists of measurements of four features of three different species of Iris flowers: Setosa, Versicolor, and Virginica

- **The four features are:**
1) Sepal length (in centimeters)
2) Sepal width (in centimeters)
3) Petal length (in centimeters)
4) Petal width (in centimeters)

- In the Iris dataset, the target variable is the species of the Iris flowers. It represents the class or category to which each sample belongs. The target variable is a categorical variable with three possible values: Setosa, Versicolor, and Virginica

- For each of the three species, there are 50 samples, resulting in a total of 150 samples in the dataset

In [0]:
# Load the iris dataset from scikit-learn
iris = load_iris()
X = iris.data
y = iris.target

In [0]:
print(X)

In [0]:
print(y)

# Part 1. Single-machine Hyperopt workflow

Here are the steps in a Hyperopt workflow:  
1. Define a function to minimize.  
2. Define a search space over hyperparameters.  
3. Select a search algorithm.  
4. Run the tuning algorithm with Hyperopt `fmin()`.

For more information, see the [Hyperopt documentation](https://github.com/hyperopt/hyperopt/wiki/FMin).

## Define a function to minimize
We use a support vector machine classifier. The objective is to find the best value for the regularization parameter `C`.  

In [0]:
def objective(C):
    # Create a support vector classifier model
    clf = SVC(C=C)
    
    # Use the cross-validation accuracy to compare the models' performance
    accuracy = cross_val_score(clf, X, y).mean()
    
    # Hyperopt tries to minimize the objective function. A higher accuracy value means a better model, so you must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}

## Define the search space over hyperparameters

In [0]:
search_space = hp.lognormal('C', 0, 1.0)

## Select a search algorithm

The two main choices are:
* `hyperopt.tpe.suggest`: Tree of Parzen Estimators, a Bayesian approach which iteratively and adaptively selects new hyperparameter settings to explore based on past results
* `hyperopt.rand.suggest`: Random search, a non-adaptive approach that samples over the search space

In [0]:
algo = tpe.suggest

## Run the tuning algorithm with Hyperopt fmin()

In [0]:
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=16
  )

In [0]:
print("Best value found: ", argmin)

# Part 2. Distributed tuning using Apache Spark and MLflow

- **Distributed tuning** A technique for tuning the hyperparameters of ML models on large datasets. It works by distributing the tuning process across multiple machines, which can significantly speed up the tuning process

- Apache Spark is a distributed computing framework that can be used to run distributed tuning jobs
- MLflow is an open-source platform for managing the end-to-end ML lifecycle, including hyperparameter tuning


To distribute tuning, add one more argument to `fmin()`: Argument `Trials` & class `SparkTrials`

`SparkTrials` takes 2 optional arguments:  
* `parallelism`: Number of models to fit and evaluate concurrently. The default is the number of available Spark task slots.
* `timeout`: Maximum time (in seconds) that `fmin()` can run. The default is no maximum time limit.

In [0]:
from hyperopt import SparkTrials

In [0]:
spark_trials = SparkTrials()

In [0]:
with mlflow.start_run():
  argmin = fmin(
    fn=objective,
    space=search_space,
    algo=algo,
    max_evals=16,
    trials=spark_trials
  )

In [0]:
print("Best value found: ", argmin)